In [1]:
import cv2
import numpy as np
import imutils
import os
import glob
from tensorflow.keras.utils import img_to_array
import tensorflow as tf
from tensorflow import keras


In [2]:
def getmodel():
    model =keras.models.load_model('../model/keras/best_model.hdf5')
    return model
a = getmodel()

OSError: No file or directory found at ../model/keras/best_model.hdf5

In [ ]:
def getmodel():
    model =keras.models.load_model('../model/keras/best_model.hdf5')
    return model

def preprocess_frame(data,img_size):
    data = imutils.resize(data, width=img_size)
    grey = cv2.cvtColor(data,cv2.COLOR_BGR2GRAY)

    return data,grey

def detectface(frame,scale=1.05, Neigh=11, mSize=(20, 20)):
    detector = cv2.CascadeClassifier('../model/haarcascade/haarcascade_frontalface_default.xml')
    return detector.detectMultiScale(frame, scaleFactor=scale, minNeighbors=Neigh, minSize=mSize,flags=cv2.CASCADE_SCALE_IMAGE)


def process_data(data,model,img_height=255,img_size=400):

    EMOTIONS = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


    frame,grey = preprocess_frame(data,img_size)    
    frameClone = frame.copy()
    canvas = np.zeros((img_height, img_size, 3), dtype="uint8")
    rects = detectface(grey)
    if len(rects) > 0:

        sort_rect = sorted(rects, reverse=False,key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))

        for o,rect in enumerate(sort_rect):        

            (fX, fY, fW, fH) = rect 

            roi = grey[fY:fY + fH, fX:fX + fW]
            roi = cv2.resize(roi, (48, 48))
            roi = roi.astype("float") / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)
            preds = model.predict(roi)[0]
            label = EMOTIONS[preds.argmax()]

            if o == 0:
                for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
                    # construct the label text
                    text = "{}: {:.2f}%".format(emotion, prob * 100)

                    # draw the label + probability bar on the canvas
                    w = int(prob * img_size)
                    cv2.rectangle(canvas, (5, (i * 35) + 5),
                    (w, (i * 35) + 35), (0, 0, 255), -1)
                    cv2.putText(canvas, text, (10, (i * 35) + 23),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45,
                    (255, 255, 255), 2, cv2.LINE_AA)
            
            text_all = f'Face {o+1} {label} {preds.max()*100:.2f}%'

            cv2.putText(frameClone, text_all, (fX, fY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2, cv2.LINE_AA)
            cv2.rectangle(frameClone, (fX, fY), (fX + fW, fY + fH),(0, 0, 255), 2)


    else:
        label = 'Notfound'
    
    return frameClone,canvas,label
    

In [ ]:
def getdata(type,input_path=None,output_path=None):

    img_size=400
    img_height=255

    model = getmodel()

    # Input is an image
    if type =='image' and input_path is not None:
        path_list  = glob.glob(input_path + '/*.jpg')
        if not(path_list):
            print('No image found please look at the path and the image extention is it jpg exit...')
            return
    
        for path in path_list:
            data = cv2.imread(path)

            pred_data,canvas,label = process_data(data,model,img_height,img_size)

            cv2.imshow("Face", pred_data)
            cv2.imshow("Probabilities", canvas)

            cv2.waitKey(0) 
            cv2.destroyAllWindows()

            filename = path.split(os.path.sep)[-1]
            filename = label + '_' + filename
            output_path = output_path+ '//'+ filename
            cv2.imwrite(filename=output_path,img=pred_data)
            
    
    # Input is  video
    else:

        if type =='video' and input_path is not None:
            path_list = glob.glob(input_path +'/*.mp4')
            if not(path_list):
                print('No video found please look at the path and the video extention is it mp4 exit...')
                return
            path = path_list[0]

            filename = path.split(os.path.sep)[-1]
            fourcc = cv2.VideoWriter_fourcc(*'MP4V')  

            output_path = output_path+ '//'+ f'output_{filename}'
            output_video = cv2.VideoWriter(output_path, fourcc, 30.0, (img_size,225))            

            camera = cv2.VideoCapture(path)

        elif type =='live':
            filename = 'live'
            fourcc = cv2.VideoWriter_fourcc(*'MP4V')  

            output_path = output_path+ '//'+ f'output_{filename}.mp4'

            output_video = cv2.VideoWriter(output_path, fourcc, 30.0, (img_size,300))

            camera = cv2.VideoCapture(0)

        else:
            print('No path pass if mode is image or video please pass path directory...')
            return 

        while True:
            grabbed,data = camera.read()

            if not grabbed:
                print('End of Video...')
                break

            pred_data,canvas,label = process_data(data,model,img_height,img_size)

            cv2.imshow("Face", pred_data)            
            cv2.imshow("Probabilities Of Face 1", canvas)

            output_video.write(pred_data)
  
            
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break

        camera.release()
        output_video.release()

        cv2.destroyAllWindows()



In [ ]:
process_type = ['image','video','live']
process_mode ={
    'image' : [ '../data/image','../output/image'],
    'video' : [ '../data/video','../output/video'],
    'live'  : [None,'../output/video']
}
mode = process_type[0]

getdata(mode,process_mode[mode][0],process_mode[mode][1]) 

In [ ]:
from tensorflow import lite
model = keras.models.load_model('../model/keras/gender_model.hdf5')
converter = lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
tflite_model_path = '../model/keras/gender_model.tflite'
with open(tflite_model_path, 'wb+') as f:
    f.write(tflite_model)


In [ ]:
from tensorflow import lite
converter = lite.TFLiteConverter.from_keras_model(a)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
tflite_model_path = '../model/keras/best_model.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)
def load_tflite_model(model_path):
    interpreter = lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    return interpreter
interpreter = load_tflite_model(tflite_model_path)

In [ ]:
def process_data(data,model,img_height=255,img_size=400):

    EMOTIONS = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


    frame,grey = preprocess_frame(data,img_size)    
    frameClone = frame.copy()
    canvas = np.zeros((img_height, img_size, 3), dtype="uint8")
    rects = detectface(grey)
    if len(rects) > 0:

        sort_rect = sorted(rects, reverse=False,key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))

        for o,rect in enumerate(sort_rect):        

            (fX, fY, fW, fH) = rect 

            roi = grey[fY:fY + fH, fX:fX + fW]
            roi = cv2.resize(roi, (48, 48))
            roi = roi.astype("float") / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)
            preds = model.predict(roi)[0]
            label = EMOTIONS[preds.argmax()]

            if o == 0:
                for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
                    # construct the label text
                    text = "{}: {:.2f}%".format(emotion, prob * 100)

                    # draw the label + probability bar on the canvas
                    w = int(prob * img_size)
                    cv2.rectangle(canvas, (5, (i * 35) + 5),
                    (w, (i * 35) + 35), (0, 0, 255), -1)
                    cv2.putText(canvas, text, (10, (i * 35) + 23),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45,
                    (255, 255, 255), 2)
            
            text_all = f'Face {o+1} {label} {preds.max()*100:.2f}%'

            cv2.putText(frameClone, text_all, (fX, fY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
            cv2.rectangle(frameClone, (fX, fY), (fX + fW, fY + fH),(0, 0, 255), 2)


    else:
        label = 'Notfound'
    
    return frameClone,canvas,label

In [ ]:
img_size=400
img_height=255
input_path = '../data/image'
model = getmodel()

path_list  = glob.glob(input_path + '/*.jpg')
path = path_list[0]



data = cv2.imread(path)

pred_data,canvas,label = process_data2(data,model,img_height,img_size)

cv2.imshow("Face", pred_data)
cv2.imshow("Probabilities", canvas)

cv2.waitKey(0) 
cv2.destroyAllWindows()

In [3]:
import cv2
import numpy as np
import imutils
import os
import glob
import argparse
from tensorflow import lite

In [22]:
[i.title() for i in ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']]

['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

In [50]:

def load_tflite_model(model_path):
    interpreter = lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    return interpreter


def inference_tflite_model(interpreter, input_data):
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Assuming the input shape is (batch_size, height, width, channels)
    input_data = input_data.reshape(input_details[0]['shape'])

    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()

    output_data = interpreter.get_tensor(output_details[0]['index'])
    return output_data

def img_toarray(img,type):   
    x = np.asarray(img, dtype="float32")
    if type=='rgb':
        x = x.reshape((x.shape[0], x.shape[1], 3))
    else:
        x = x.reshape((x.shape[0], x.shape[1], 1))

    return np.expand_dims(x, axis=0)

def preprocess_frame(data):
    input_mood = cv2.resize(data, dsize=(48,48))
    input_mood = cv2.cvtColor(input_mood,cv2.COLOR_BGR2GRAY).astype("float") / 255.0
    input_age = cv2.resize(data, dsize=(200,200)).astype("float") / 255.0
    input_gender = cv2.resize(data, dsize=(110,90)).astype("float") / 255.0

    input_mood = img_toarray(input_mood,'grey')
    input_age = img_toarray(input_age,'rgb')
    input_gender = img_toarray(input_gender,'rgb')


    return input_mood,input_age,input_gender

def detectface(model_path):
    detector = cv2.CascadeClassifier(model_path)
    return detector


def process_data(data,img_height=255,img_size=400):

    age_model = '../model/keras/age_model.tflite'
    mood_model = '../model/keras/mood_model.tflite'
    gender_model = '../model/keras/gender_model.tflite'
    detector_path = '../model/haarcascade/haarcascade_frontalface_default.xml'

    EMOTIONS = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
    AGE = ['0-2', '13-20', '21-32', '33-48', '3-6', '49-53', '54+', '7-12']
    GENDER = ['FEMALE','MALE']

    # frame,grey = preprocess_frame(data,img_size,cv2.COLOR_BGR2GRAY)    
    data = imutils.resize(data,img_size)
    # frameClone = frame.copy()
    canvas = np.zeros((img_height, img_size, 3), dtype="uint8")
    detector =detectface(detector_path)

    rects = detector.detectMultiScale(data, 
                                      scaleFactor=1.05, minNeighbors=11, 
                                      minSize=(20, 20),flags=cv2.CASCADE_SCALE_IMAGE)
    if len(rects) > 0:
        
        

        age_model = load_tflite_model(age_model)
        gender_model = load_tflite_model(gender_model)
        mood_model = load_tflite_model(mood_model)

        # frame,input_mood,input_age,input_gender = preprocess_frame(data)    
        frameClone = data.copy()
        sort_rect = sorted(rects, reverse=False,key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))

        for o,rect in enumerate(sort_rect):        

            (fX, fY, fW, fH) = rect 

            roi = data[fY:fY + fH, fX:fX + fW]
            
            input_mood,input_age,input_gender = preprocess_frame(roi)

            # roi = cv2.resize(roi, (48, 48))
            # roi = roi.astype("float") / 255.0
            # roi = img_toarray(roi)
            # roi = np.expand_dims(roi, axis=0)
            # preds = model.predict(roi)[0]

            preds_mood = inference_tflite_model(mood_model,input_mood)[0]
            preds_age = inference_tflite_model(age_model,input_age)[0]
            # preds_gender = inference_tflite_model(gender_model,input_gender)[0]
            
            label = EMOTIONS[preds_mood.argmax()]
            label_age = AGE[preds_age.argmax()]
            # label_gender = GENDER[preds_gender.argmax()]

            if o == 0:
                for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds_mood)):
                    # construct the label text
                    text = "{}: {:.2f}%".format(emotion, prob * 100)

                    # draw the label + probability bar on the canvas
                    w = int(prob * img_size)
                    cv2.rectangle(canvas, (5, (i * 35) + 5),
                    (w, (i * 35) + 35), (0, 0, 255), -1)
                    cv2.putText(canvas, text, (10, (i * 35) + 23),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45,
                    (255, 255, 255), 1, cv2.LINE_AA)
            
            text_all = f'Face {o+1} {label} {preds_mood.max()*100:.2f}% {label_age}'
            pos = o+1
            # text_all = f'Face {pos} {label} {label_age}'

            if (o % 2) == 0:
                cv2.putText(frameClone, text_all, (fX, fY - (pos*10)), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 1, cv2.LINE_AA)
            else:
                cv2.putText(frameClone, text_all, (fX, fY + fH + (pos*10)), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 1, cv2.LINE_AA)
            cv2.rectangle(frameClone, (fX, fY), (fX + fW, fY + fH),(0, 0, 255), 2)


    else:
        label = 'Notfound'
        frameClone = data
    
    return frameClone,canvas,label
    

def getdata(type,input_path=None,output_path=None):

    img_size=400
    img_height=255

    # model = getmodel(model_path)

    # Input is an image
    if type =='image' and input_path is not None:
        path_list  = glob.glob(input_path + '/*.jpg')
        if not(path_list):
            print('No image found please look at the path and the image extention is it jpg exit...')
            return
    
        for path in path_list:
            data = cv2.imread(path)

            pred_data,canvas,label = process_data(data,img_height,img_size)

            cv2.imshow("Face", pred_data)
            cv2.imshow("Probabilities", canvas)

            cv2.waitKey(0) 
            cv2.destroyAllWindows()

            filename = path.split(os.path.sep)[-1]
            filename = label + '_' + filename
            output_path = output_path+ '//'+ filename
            cv2.imwrite(filename=output_path,img=pred_data)
            
    
    # Input is  video
    else:

        if type =='video' and input_path is not None:
            path_list = glob.glob(input_path +'/*.mp4')
            if not(path_list):
                print('No video found please look at the path and the video extention is it mp4 exit...')
                return
            path = path_list[0]

            filename = path.split(os.path.sep)[-1]
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')  

            output_path = output_path+ '//'+ f'output_{filename}'
            output_video = cv2.VideoWriter(output_path, fourcc, 60.0, (img_size,225))            

            camera = cv2.VideoCapture(path)

        elif type =='live':
            filename = 'live'
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')  

            output_path = output_path+ '//'+ f'output_{filename}.mp4'

            output_video = cv2.VideoWriter(output_path, fourcc, 60.0, (img_size,300))

            camera = cv2.VideoCapture(0)

        else:
            print('No path pass if mode is image or video please pass path directory...')
            return 

        while True:
            grabbed,data = camera.read()

            if not grabbed:
                print('End of Video...')
                break

            pred_data,canvas,label = process_data(data,img_height,img_size)

            cv2.imshow("Face", pred_data)            
            cv2.imshow("Probabilities Of Face 1", canvas)

            output_video.write(pred_data)
  
            
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break

        camera.release()
        output_video.release()

        cv2.destroyAllWindows()

        print('Script finish...')


In [51]:
a = getdata('live',None,'../output/video')

Script finish...
